In [1]:
import site
site.getsitepackages()

['/Users/songyujian/PycharmProjects/intern/venv/lib/python3.8/site-packages']

In [2]:
import sys
sys.path.append('/Users/songyujian/anaconda3/lib/python3.11/site-packages')

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
import codecs

In [4]:
def dummies_Sp(data,list,_sep):
    df_tmp = data[list].str.get_dummies(sep=_sep).add_prefix(list+'_')
    data = pd.concat([data, df_tmp], axis=1)
    data = data.drop(columns=[list])
    return data


In [5]:
f = codecs.open('log.txt','w','utf-8')

ExcelPath = './data.xlsx'
data = pd.read_excel(ExcelPath,sheet_name='Sheet1', skiprows=lambda x: x in [0])

ListX = ['logo出现时长_解析','品牌提及次数','明星系数','广告语字数']

ListY = ['品牌回忆指数']

X = data[ListX]
Y = data[ListY]


X = pd.get_dummies(X)

min_Y, max_Y = Y[list].min(), Y[list].max()  # 求出'C'列数据的最小值和最大值
Y[list] = (Y[list] - min_Y[0]) / (max_Y[0] - min_Y[0])  # 进行最小-最大标准化
Y[list] = Y[list].apply(np.log1p)


from sklearn.model_selection import train_test_split

# 随机抽取训练集和验证集
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3, random_state=0)
Xtrain.to_excel('Xtrain.xlsx')
Xtest.to_excel('Xtest.xlsx')
print('train,test已保存')

/var/folders/_h/khft8pzs5tjcns8cn97x2x3m0000gn/T/ipykernel_14010/1742304880.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y[list] = (Y[list] - min_Y[0]) / (max_Y[0] - min_Y[0])  # 进行最小-最大标准化
/var/folders/_h/khft8pzs5tjcns8cn97x2x3m0000gn/T/ipykernel_14010/1742304880.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y[list] = Y[list].apply(np.log1p)


train,test已保存


In [ ]:
train_matrix = xgb.DMatrix(data=Xtrain, label=Ytrain)

best_mae = 1
best_mae_index = 0

# 有明确调优后的参数用这个，并且不需要循环
# best_param={'max_depth': 6, 'eta': 0.09, 'n_estimators': 100, 'gamma': 0.2, 'subsample': 0.65, 'colsample_bytree': 0.6, 'min_child_weight': 7, 'max_delta_step': 9, 'seed': 33, 'objective': 'reg:logistic', 'eval_metric': 'mae'}
# for i in range(1):

# 调参验证  有明确参数以后可以不需要调参
best_param={}
for i in range(10000):
    params = {
        'max_depth': np.random.randint(5, 16),
        'eta': np.random.uniform(0.01, 0.3),
        'n_estimators': np.random.randint(10, 300),
        'gamma': np.random.uniform(0.0, 0.2),
        'subsample': np.random.uniform(0.6, 0.9),
        'colsample_bytree': np.random.uniform(0.5, 0.8),
        'min_child_weight': np.random.randint(1, 41),
        'max_delta_step': np.random.randint(1, 11),
        'seed': 2023,
        'objective':'reg:logistic',
        'eval_metric':'mae'
    }
    params = best_param
    # eta ： 默认是0.3，别名是 leanring_rate，更新过程中用到的收缩步长，在每次提升计算之后，算法会直接获得新特征的权重。 eta通过缩减特征的权重使提升计算过程更加保守；
    # gamma：默认是0，别名是 min_split_loss，在节点分裂时，只有在分裂后损失函数的值下降了（达到gamma指定的阈值），才会分裂这个节点。gamma值越大，算法越保守（越不容易过拟合）；
    # max_depth：默认是6，树的最大深度，值越大，越容易过拟合；
    # min_child_weight：默认是1，决定最小叶子节点样本权重和，加权和低于这个值时，就不再分裂产生新的叶子节点。当它的值较大时，可以避免模型学习到局部的特殊样本。但如果这个值过高，会导致欠拟合。
    # max_delta_step：默认是0，这参数限制每颗树权重改变的最大步长。如果是 0 意味着没有约束。如果是正值那么这个算法会更保守，通常不需要设置。
    # subsample：默认是1，这个参数控制对于每棵树，随机采样的比例。减小这个参数的值算法会更加保守，避免过拟合。但是这个值设置的过小，它可能会导致欠拟合。
    # colsample_bytree：默认是1，用来控制每颗树随机采样的列数的占比；
    # colsample_bylevel：默认是1，用来控制的每一级的每一次分裂，对列数的采样的占比；
    # lambda：默认是1，别名是reg_lambda，L2 正则化项的权重系数，越大模型越保守；
    # alpha：默认是0，别名是reg_alpha，L1 正则化项的权重系数，越大模型越保守；
    # seed：随机数种子，相同的种子可以复现随机结果，用于调参！
    # n_estimators：弱学习器的数量

    cv_nround = 1000
    cv_nfold = 5
    xgb_cv = xgb.cv(dtrain=train_matrix, params=params, num_boost_round=cv_nround, nfold=cv_nfold,
                    verbose_eval=True, early_stopping_rounds=5, maximize=False, metrics='mae')

    min_mae_index = xgb_cv['test-mae-mean'].idxmin()
    min_mae = xgb_cv['test-mae-mean'][min_mae_index]

    print('min_mae_index',min_mae_index)
    print('min_mae',min_mae)

    if min_mae < best_mae:
        best_mae = min_mae
        best_mae_index = min_mae_index
        best_param = params


nround = best_mae_index

print(best_param)

f.write(str(best_param)+ '\r\n')
f.close()


bst_model = xgb.train(params=best_param, dtrain=train_matrix, num_boost_round=nround)
bst_model.save_model('AI_BL_xgb.model')
print('AI_BL_xgb已保存')

[0]	train-mae:0.07303+0.00087	test-mae:0.07797+0.00314
[1]	train-mae:0.06858+0.00107	test-mae:0.07738+0.00321
[2]	train-mae:0.06549+0.00117	test-mae:0.07712+0.00363
[3]	train-mae:0.06294+0.00133	test-mae:0.07777+0.00376
[4]	train-mae:0.06089+0.00112	test-mae:0.07775+0.00406
[5]	train-mae:0.05891+0.00094	test-mae:0.07805+0.00404
[6]	train-mae:0.05752+0.00064	test-mae:0.07840+0.00397
[7]	train-mae:0.05601+0.00078	test-mae:0.07848+0.00408
min_mae_index 2
min_mae 0.07711680841445923
[0]	train-mae:0.07303+0.00087	test-mae:0.07797+0.00314
[1]	train-mae:0.06858+0.00107	test-mae:0.07738+0.00321
[2]	train-mae:0.06549+0.00117	test-mae:0.07712+0.00363
[3]	train-mae:0.06294+0.00133	test-mae:0.07777+0.00376
[4]	train-mae:0.06089+0.00112	test-mae:0.07775+0.00406
[5]	train-mae:0.05891+0.00094	test-mae:0.07805+0.00404
[6]	train-mae:0.05752+0.00064	test-mae:0.07840+0.00397
[7]	train-mae:0.05601+0.00078	test-mae:0.07848+0.00408
min_mae_index 2
min_mae 0.07711680841445923
[0]	train-mae:0.07303+0.00087	te